In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
import pandas as pd

raw_df = pd.read_csv('./raw_keywords.csv')[['Keywords', 'Search Volume', 'Competing Products']]

raw_df['Competing Products'] = raw_df['Competing Products'].fillna(0)
raw_df['Competing Products'] = raw_df['Competing Products'].str.replace('>', '').astype('Int64')
raw_df

,Keywords,Search Volume,Competing Products
0,zaanzeer chicken shredder tool,23.0,231
1,amazon chicken shredder,432.0,315
2,aparato para desmenuzar el pollo,90.0,167
3,avefare chicken shredder tool twist,37.0,214
4,baked chicken shredder,122.0,250
...,...,...,...
195,top selling items on amazon,439.0,30000
196,under $100,92.0,727
197,vegetable chopper small,290.0,5000
198,wedding registry gifts for men,119.0,353


In [ ]:
import pandas as pd
import re
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 상태 정의
class State(MessagesState):
    data: pd.DataFrame

llm = ChatOpenAI(model="gpt-5-mini")  

# ResponseSchema 정의
response_schemas = [
    ResponseSchema(
        name="keywords",
        description="조건을 적용한 키워드 리스트, 각 키워드는 dict 형태로 유지"
    )
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트
template = """
다음 키워드 리스트에 대해 조건을 적용하되, 삭제는 가능한 한 신중히 수행한다:

[조건]
1. 영어 이외의 외국어 키워드 삭제 권장. 
2. 같은 키워드의 단수형과 복수형 단어가 동시에 존재할 경우에만, 복수형 키워드를 제거함 
3. 오타로 판단되는 키워드의 경우, 원본 키워드가 존재할 경우 오타 키워드를 제거함


[데이터]
{data}

[출력]
{format_instructions}
"""

def preprocess_keywords(df: pd.DataFrame) -> pd.DataFrame:
    """Python에서 사전 필터링: 영어, 알파벳/숫자만"""
    
    df = df[df['Keywords'].apply(lambda x: bool(re.fullmatch(r"[A-Za-z0-9 &'().-]+", str(x))))]    
    df = df.sort_values('Search Volume', ascending=False).drop_duplicates(subset=['Keywords'])
    
    return df.reset_index(drop=True)

def keyword_process(state: State):
    df = state["data"]
    
    # 1. 사전 필터링
    df = preprocess_keywords(df)
    
    # 2. LLM 호출
    prompt = PromptTemplate.from_template(template)
    keyword_prompt = prompt.format(
        data=df.to_dict(orient="records"),
        format_instructions=parser.get_format_instructions()
    )
    
    res = llm.invoke([{"role": "user", "content": keyword_prompt}])
    structured = parser.parse(res.content)
    
    cleaned_data = pd.DataFrame(structured["keywords"])
    
    return {"data": cleaned_data}

# 그래프 정의
builder = StateGraph(State).add_node("keyword_process", keyword_process)
builder.add_edge(START, "keyword_process")

graph = builder.compile()

# 실행 예시
final_state = graph.invoke({
    "data": raw_df,
})

df = final_state["data"]
 

In [22]:
from sklearn.preprocessing import StandardScaler

scale_cols = ['Search Volume', 'Competing Products']

ss = StandardScaler()
scaled_array = ss.fit_transform(df[scale_cols])
scaled_df = pd.DataFrame(scaled_array, columns=scale_cols)
cleaned_df = pd.concat([df['Keywords'], scaled_df], axis=1)
cleaned_df['value_score'] = ((cleaned_df['Search Volume'] + 1) / (cleaned_df['Competing Products'] + 1))

cleaned_df.to_csv('./processed_keywords.csv')

In [ ]:
import pandas as pd
import re
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 상태 정의
class State(MessagesState):
    data: pd.DataFrame
    deleted_keywords: list = []  # 삭제된 키워드 저장

llm = ChatOpenAI(model="gpt-5-mini")  

# ResponseSchema 정의
response_schemas = [
    ResponseSchema(
        name="keywords",
        description="조건을 적용한 키워드 리스트, 각 키워드는 dict 형태로 유지"
    )
]
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트
template = """
다음 키워드 리스트에 대해 조건을 적용하되, 삭제는 가능한 한 신중히 수행한다:

[조건]
1. 영어 이외의 외국어 키워드 삭제 권장. 
2. 같은 키워드의 단수형과 복수형 단어가 동시에 존재할 경우에만, 복수형 키워드를 제거함 
3. 오타로 판단되는 키워드의 경우, 원본 키워드가 존재할 경우 오타 키워드를 제거함

[데이터]
{data}

[출력]
{format_instructions}
"""

def preprocess_keywords(df: pd.DataFrame) -> pd.DataFrame:
    """Python에서 사전 필터링: 영어, 알파벳/숫자만"""
    df = df[df['Keywords'].apply(lambda x: bool(re.fullmatch(r"[A-Za-z0-9 &'().-]+", str(x))))]    
    df = df.sort_values('Search Volume', ascending=False).drop_duplicates(subset=['Keywords'])
    return df.reset_index(drop=True)

def keyword_process(state: State):
    df = state["data"]
    
    # 1. 사전 필터링
    filtered_df = preprocess_keywords(df[['Keywords', 'Search Volume']])
    
    # 2. LLM 호출
    prompt = PromptTemplate.from_template(template)
    keyword_prompt = prompt.format(
        data=filtered_df.to_dict(orient="records"),
        format_instructions=parser.get_format_instructions()
    )
    
    res = llm.invoke([{"role": "user", "content": keyword_prompt}])
    structured = parser.parse(res.content)
    
    # 3. LLM 결과 키워드 추출
    cleaned_keywords = [k['Keywords'] for k in structured["keywords"]]
    
    # 4. 삭제된 키워드 추출
    deleted_keywords = df[~df['Keywords'].isin(cleaned_keywords)]['Keywords'].tolist()
    
    # 5. 원본 df에서 LLM 통과한 키워드만 필터
    cleaned_data = df[df['Keywords'].isin(cleaned_keywords)].reset_index(drop=True)
    
    return {
        "data": cleaned_data,
        "deleted_keywords": deleted_keywords
    }

# 그래프 정의
builder = StateGraph(State).add_node("keyword_process", keyword_process)
builder.add_edge(START, "keyword_process")

graph = builder.compile()

# 실행 예시
# raw_df 는 원본 데이터프레임
final_state = graph.invoke({
    "data": raw_df,
})

df_cleaned = final_state["data"]
deleted_list = final_state["deleted_keywords"]

print("필터링 후 데이터프레임:")
print(df_cleaned)
print("\n삭제된 키워드 리스트:")
print(deleted_list)